In [43]:
from openai import OpenAI
import os

client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role":"system","content":"You are a helpfull assistant. Answer the questions in a very friendly manner."},
    {"role":"user","content":"Why should i learn open AI API?"}
  ]
)
completion.choices[0].message.content

'Learning OpenAI API can be a great opportunity for you to explore the world of artificial intelligence and machine learning. By mastering OpenAI API, you can create advanced AI models, develop innovative applications, and contribute to cutting-edge research in the field. It can also open up new career opportunities and help you stay updated with the latest trends in technology. So, go ahead and dive into learning OpenAI API - you never know where it might lead you!'

In [48]:
import pandas as pd
import sqlalchemy

from sqlalchemy import create_engine
from sqlalchemy import text


In [47]:
df = pd.read_csv('car-mpg.csv')

In [49]:
temp_db = create_engine('sqlite:///:memory:',echo=True)

In [50]:
data = df.to_sql('car',con=temp_db)

2024-05-04 12:42:32,309 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-04 12:42:32,324 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("car")
2024-05-04 12:42:32,327 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-04 12:42:32,331 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("car")
2024-05-04 12:42:32,333 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-04 12:42:32,343 INFO sqlalchemy.engine.Engine 
CREATE TABLE car (
	"index" BIGINT, 
	mpg FLOAT, 
	cyl BIGINT, 
	disp FLOAT, 
	hp TEXT, 
	wt BIGINT, 
	acc FLOAT, 
	yr BIGINT, 
	origin BIGINT, 
	car_type BIGINT, 
	car_name TEXT
)


2024-05-04 12:42:32,345 INFO sqlalchemy.engine.Engine [no key 0.00173s] ()
2024-05-04 12:42:32,351 INFO sqlalchemy.engine.Engine CREATE INDEX ix_car_index ON car ("index")
2024-05-04 12:42:32,363 INFO sqlalchemy.engine.Engine [no key 0.01224s] ()
2024-05-04 12:42:32,399 INFO sqlalchemy.engine.Engine INSERT INTO car ("index", mpg, cyl, disp, hp, wt, acc, yr, origin, car_type, car_name) V

In [52]:
with temp_db.connect() as conn:
    result = conn.execute(text("SELECT * FROM CAR"))

2024-05-04 12:44:41,308 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-04 12:44:41,310 INFO sqlalchemy.engine.Engine SELECT * FROM CAR
2024-05-04 12:44:41,313 INFO sqlalchemy.engine.Engine [cached since 19.61s ago] ()
2024-05-04 12:44:41,317 INFO sqlalchemy.engine.Engine ROLLBACK


In [130]:
def create_system_prompt(df):
    
    prompt = """You are an SQL expert. Here is sqlite SQL table, with their properties:
    
    car({}) 
                
your job is to write queries given a users request. Only Output the SQL query.
Start with SELECT statement""".format(",".join(df.columns.tolist()))
    return prompt

In [131]:
def create_user_prompt():
    q = input("Enter your text query: ")
    return q

In [132]:
sys = create_system_prompt(df)
usr = create_user_prompt()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role":"system","content":f"{sys}"},
    {"role":"user","content":f"{usr}"}
  ],
  stop = [';']
)


In [134]:
print(completion.choices[0].message.content)

SELECT car_name, COUNT(car_name) AS appearance_count
FROM car
GROUP BY car_name
ORDER BY appearance_count DESC
LIMIT 1


In [135]:
with temp_db.connect() as conn:
    result = conn.execute(text(completion.choices[0].message.content))
print(result.all())

2024-05-04 13:45:45,465 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-04 13:45:45,466 INFO sqlalchemy.engine.Engine SELECT car_name, COUNT(car_name) AS appearance_count
FROM car
GROUP BY car_name
ORDER BY appearance_count DESC
LIMIT 1
2024-05-04 13:45:45,467 INFO sqlalchemy.engine.Engine [generated in 0.00241s] ()
2024-05-04 13:45:45,476 INFO sqlalchemy.engine.Engine ROLLBACK
[('ford pinto', 6)]


[]